In [ ]:
%pip install selenium
%pip install beautifulsoup4
%pip install plotly
%pip install seaborn
%pip install scipy
%pip install nbformat

In [71]:
%pip freeze | findstr nbformat


nbformat==5.10.4
Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [2]:
from selenium.webdriver.chrome.service import Service

download_service = Service()

driver = webdriver.Chrome(service=download_service)

#driver.maximize_window()

The chromedriver version (131.0.6778.69) detected in PATH at I:\ChromeDriver\chromedriver.exe might not be compatible with the detected chrome version (132.0.6834.110); currently, chromedriver 132.0.6834.110 is recommended for chrome 132.*, so it is advised to delete the driver in PATH and retry


In [3]:
# Otwieranie strony Steam
url = "https://store.steampowered.com/category/science_fiction/"
driver.get(url)

# Czekanie na załadowanie treści
time.sleep(2)  # Dostosuj, jeśli strona ładuje się szybciej

In [10]:
driver.execute_script('window.scroll(0, document.body.scrollHeight-2000)')


In [29]:
load_more_button = driver.find_element(By.CLASS_NAME, '_2tkiJ4VfEdI9kq1agjZyNz.Focusable')
load_more_button.click()

In [4]:
# todo: wyciągnąć jeszcze: datę wydania, opinie użytkownikow i liczbę opinii

def GetTitlesWithTagsAndPrices():
    games = []
    game_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'v9uRg57bwOaPsvAnkXESO')]")

    for game in game_containers:
        try:
            # Pobranie tytułu gry
            title = game.find_element(By.XPATH, ".//div[contains(@class, '_3rrH9dPdtHVRMzAEw82AId')]//a//div").text.strip()

            # Pobranie tagów
            tags_container = game.find_element(By.XPATH, ".//div[contains(@class, '_3wryhCRrTuMULeq_YjNk-s')]")
            tag_elements = tags_container.find_elements(By.XPATH, ".//div[contains(@class, '_2bkP-3b7dvr0a_qPdZEfHY')]//a")
            tags = [tag.text.strip() for tag in tag_elements if tag.text.strip()]

            # Pobranie ceny gry
            try:
                price_container = game.find_element(By.XPATH, ".//div[contains(@class, '_1JuIpzMtS7-xZrnUmEQ4my')]//div[contains(@class, '_2s-O5T3qJJYR2AUq4b9jIN')]")
                price = price_container.find_element(By.XPATH, ".//div[contains(@class, '_3j4dI1yA7cRfCvK8h406OB')]").text.strip()
            except Exception as e:
                price = "Brak ceny"  # Obsługa gier bez podanej ceny

            # Dodanie gry do listy
            games.append({
                "title": title,
                "tags": tags,
                "price": price
            })
        except Exception as e:
            print(f"Problem z przetwarzaniem gry: {e}")

    # Wyświetlenie wyników
    print("Lista gier, ich tagów i cen:")
    for game in games:
        print(f"Tytuł: {game['title']}, Tagi: {', '.join(game['tags'])}, Cena: {game['price']}")

    return games


In [9]:
import pandas as pd

def GetTitlesWithDetails():
    games_data = []
    game_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'v9uRg57bwOaPsvAnkXESO')]")

    for game in game_containers:
        try:
            # Pobranie tytułu gry
            title = game.find_element(By.XPATH, ".//div[contains(@class, '_3rrH9dPdtHVRMzAEw82AId')]//a//div").text.strip()

            # Pobranie tagów
            tags_container = game.find_element(By.XPATH, ".//div[contains(@class, '_3wryhCRrTuMULeq_YjNk-s')]")
            tag_elements = tags_container.find_elements(By.XPATH, ".//div[contains(@class, '_2bkP-3b7dvr0a_qPdZEfHY')]//a")
            tags = ', '.join([tag.text.strip() for tag in tag_elements if tag.text.strip()])

            # Pobranie ceny gry
            try:
                price_container = game.find_element(By.XPATH, ".//div[contains(@class, 'kW6m4Sjqacp5hykrj5LEo')]//div[contains(@class, '_2s-O5T3qJJYR2AUq4b9jIN')]")
                price = price_container.find_element(By.XPATH, ".//div[contains(@class, '_3j4dI1yA7cRfCvK8h406OB')]").text.strip()
            except:
                price = "Free to Play"

            # Pobranie daty wydania
            try:
                release_date = game.find_element(By.XPATH, ".//div[contains(@class, '_3wryhCRrTuMULeq_YjNk-s')]//div[contains(@class, '_3a6HRK-P6LK0-pxRKXYgyP')]//div[contains(@class, '_1qvTFgmehUzbdYM9cw0eS7')]").text.strip()
            except:
                release_date = "Brak daty"

            # Pobranie nacechowania recenzji
            try:
                review_sentiment = game.find_element(By.XPATH, ".//div[contains(@class, '_3wryhCRrTuMULeq_YjNk-s')]//a[contains(@class, '_3qvppfM_u0yn2jrpoUo8RM')]//div[contains(@class, '_3ZWs0kB-1tuqQtie9KK-E7')]//div[contains(@class, '_2nuoOi5kC2aUI12z85PneA')]").text.strip()
            except:
                review_sentiment = "Brak informacji"

            # Pobranie liczby recenzji
            try:
                review_count = game.find_element(By.XPATH, ".//div[contains(@class, '_3wryhCRrTuMULeq_YjNk-s')]//a[contains(@class, '_3qvppfM_u0yn2jrpoUo8RM')]//div[contains(@class, '_3ZWs0kB-1tuqQtie9KK-E7')]//div[contains(@class, '_1wXL_MfRpdKQ3wZiNP5lrH')]").text.strip()
            except:
                review_count = "0"

            # Dodanie gry do listy
            games_data.append({
                "Tytuł": title,
                "Tagi": tags,
                "Cena": price,
                "Data wydania": release_date,
                "Nacechowanie recenzji": review_sentiment,
                "Liczba recenzji": review_count
            })

        except Exception as e:
            print(f"Problem z przetwarzaniem gry: {e}")

    # Konwersja do DataFrame
    df = pd.DataFrame(games_data)
    return df


In [13]:
data = GetTitlesWithDetails()

In [5]:
# zescrolluj do początku listy
#driver.execute_script('window.scroll(0, document.body.scrollHeight-3000)')

# zecrolluj na dół listy
driver.execute_script('window.scroll(0, document.body.scrollHeight-2000)')

# Powtarzanie procesu 10 razy
for i in range(25):
    load_more_button = driver.find_element(By.CLASS_NAME, '_2tkiJ4VfEdI9kq1agjZyNz.Focusable')
    load_more_button.click()
    time.sleep(2)
    driver.execute_script('window.scroll(0, document.body.scrollHeight-2000)')


data = GetTitlesWithTagsAndPrices()

Lista gier, ich tagów i cen:
Tytuł: Marvel Rivals, Tagi: Free to Play, Wieloosobowe, Strzelanka z bohaterami, Cena: Free to Play
Tytuł: Warhammer 40,000: Space Marine 2, Tagi: Warhammer 40K, Akcja, Strzelanka z perspektywą trzeciej osoby, Wieloosobowe, Cena: 157,49 zł
Tytuł: Detroit: Become Human, Tagi: Znaczące wybory, Bogata fabuła, Wiele zakończeń, Filmowe, Jednoosobowe, Cena: 55,49 zł
Tytuł: Stellaris, Tagi: Kosmos, Strategiczne globalne, Strategiczne, Science fiction, 4X, Cena: 46,24 zł
Tytuł: HELLDIVERS™ 2, Tagi: Sieciowa kooperacja, PvE, Strzelanka z perspektywą trzeciej osoby, Wieloosobowe, Akcja, Cena: 169,00 zł
Tytuł: Cyberpunk 2077, Tagi: Cyberpunk, Otwarty świat, Nagość, RPG, Jednoosobowe, Cena: 199,00 zł
Tytuł: Among Us, Tagi: Wieloosobowe, Sieciowa kooperacja, Dedukcyjne towarzyskie, Kosmos, Survival, Cena: 10,79 zł
Tytuł: Apex Legends™, Tagi: Free to Play, Battle royale, Wieloosobowe, FPS, Strzelanka, Cena: Free to Play
Tytuł: Warframe, Tagi: Free to Play, RPG akcji, Str

In [6]:
# Zamknięcie drivera
driver.quit()

In [12]:
import pandas as pd
import numpy as np

def clean_data(df):
    # 1. Konwersja ceny
    def clean_price(price):
        try:
            if "Free to Play" in price:
                return 0.0
            return float(price.replace(' zł', '').replace(',', '.'))
        except:
            return np.nan

    df['Cena'] = df['Cena'].apply(clean_price)

    # 2. Konwersja liczby recenzji
    def clean_reviews(reviews):
        try:
            return int(reviews.replace('Recenzje użytkowników: ', '').replace(' ', ''))
        except:
            return np.nan

    df['Liczba recenzji'] = df['Liczba recenzji'].apply(clean_reviews)

    # 3. Konwersja daty wydania
    polish_months = {
        'STY': 'Jan', 'LUT': 'Feb', 'MAR': 'Mar', 'KWI': 'Apr', 'MAJ': 'May', 'CZE': 'Jun',
        'LIP': 'Jul', 'SIE': 'Aug', 'WRZ': 'Sep', 'PAŹ': 'Oct', 'LIS': 'Nov', 'GRU': 'Dec'
    }

    def clean_date(date):
        if pd.isna(date):
            return np.nan
        for pl, en in polish_months.items():
            date = date.replace(pl, en)
        try:
            return pd.to_datetime(date, format='%d %b %Y')
        except:
            return np.nan

    df['Data wydania'] = df['Data wydania'].apply(clean_date)

    # 4. Rozbicie tagów na kolumny (one-hot encoding)
    def split_tags(tags):
        return tags.split(', ') if isinstance(tags, str) else []

    all_tags = sorted(set(tag for tags in df['Tagi'].apply(split_tags) for tag in tags))
    for tag in all_tags:
        df[f'Tag_{tag}'] = df['Tagi'].apply(lambda tags: 1 if tag in split_tags(tags) else 0)

    # 5. Obsługa braków danych
    df.dropna(subset=['Cena', 'Liczba recenzji', 'Data wydania'], inplace=True)

    return df


In [15]:
display(df)

,Tytuł,Tagi,Cena,Data wydania,Nacechowanie recenzji,Liczba recenzji,Tag_4X,Tag_Akcja,Tag_Battle royale,Tag_Bogata fabuła,...,Tag_Strzelanka z łupami,Tag_Superbohater,Tag_Survival,Tag_Survival horror,Tag_Survivalowe w otwartym świecie,Tag_Warhammer 40K,Tag_Wiele zakończeń,Tag_Wieloosobowe,Tag_Zarządzanie zasobami,Tag_Znaczące wybory
0,Marvel Rivals,"Free to Play, Wieloosobowe, Strzelanka z bohat...",0.00,2024-12-06,Bardzo pozytywne,179402,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,"Warhammer 40,000: Space Marine 2","Warhammer 40K, Akcja, Strzelanka z perspektywą...",157.49,2024-09-09,Bardzo pozytywne,107725,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0
2,Detroit: Become Human,"Znaczące wybory, Bogata fabuła, Wiele zakończe...",55.49,2020-06-18,Przytłaczająco pozytywne,129150,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
3,Among Us,"Wieloosobowe, Sieciowa kooperacja, Dedukcyjne ...",10.79,2018-11-16,Bardzo pozytywne,611949,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,HELLDIVERS™ 2,"Sieciowa kooperacja, PvE, Strzelanka z perspek...",169.00,2024-02-08,W większości pozytywne,706128,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
5,Apex Legends™,"Free to Play, Battle royale, Wieloosobowe, FPS...",0.00,2020-11-05,Mieszane,976121,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
6,Stellaris,"Kosmos, Strategiczne globalne, Strategiczne, S...",46.24,2016-05-09,Bardzo pozytywne,124220,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Cyberpunk 2077,"Cyberpunk, Otwarty świat, Nagość, RPG, Jednoos...",199.00,2020-12-10,Bardzo pozytywne,715099,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,ICARUS,"Survival, Survivalowe w otwartym świecie, Wiel...",40.49,2021-12-03,W większości pozytywne,35280,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
10,Frostpunk,"Budowanie miasta, Survival, Zarządzanie zasoba...",13.89,2018-04-24,Bardzo pozytywne,98112,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [14]:
import pandas as pd

# Dane wejściowe
df = pd.DataFrame(data)

# Przetwarzanie danych
df = clean_data(df)


In [10]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import plotly.express as px

def analyze_data(df):
    # 1. Tworzenie tabeli krzyżowej tagów
    def create_crosstab(df):
        all_tags = [tag for tags in df['tags'] for tag in tags]  # Zbiór wszystkich tagów
        unique_tags = sorted(set(all_tags))  # Unikalne tagi
        
        # Tworzenie macierzy obecności tagów
        tag_matrix = pd.DataFrame(0, index=unique_tags, columns=unique_tags)
        for tags in df['tags']:
            for tag1 in tags:
                for tag2 in tags:
                    if tag1 != tag2:  # Ignorujemy współwystępowanie z samym sobą
                        tag_matrix.loc[tag1, tag2] += 1
        
        return tag_matrix

    # 2. Test chi-kwadrat na współwystępowanie tagów
    def chi_square_analysis(tag_matrix):
        chi2_results = []
        for tag1 in tag_matrix.index:
            for tag2 in tag_matrix.columns:
                if tag1 != tag2:  # Test tylko dla różnych tagów
                    obs = [[tag_matrix.loc[tag1, tag2], tag_matrix.loc[tag2, tag1]],
                           [sum(tag_matrix.loc[tag1, :]), sum(tag_matrix.loc[:, tag2])]]
                    _, p, _, _ = chi2_contingency(obs)
                    chi2_results.append((tag1, tag2, p))
        
        return pd.DataFrame(chi2_results, columns=['Tag1', 'Tag2', 'P-value']).sort_values('P-value')

    # 3. Analiza wpływu tagów na ceny
    def tag_price_analysis(df):
        tag_price_data = []
        for tag in sorted(set(tag for tags in df['tags'] for tag in tags)):
            filtered = df[df['tags'].apply(lambda tags: tag in tags)]
            avg_price = filtered['price'].mean()
            tag_price_data.append({'Tag': tag, 'Average Price': avg_price})
        
        return pd.DataFrame(tag_price_data).sort_values('Average Price', ascending=False)

    # Tworzenie tabeli krzyżowej tagów
    tag_matrix = create_crosstab(df)
    print("Tabela krzyżowa tagów:\n", tag_matrix)

    # Analiza współwystępowania tagów z testem chi-kwadrat
    #chi2_results = chi_square_analysis(tag_matrix)
    #print("Wyniki testu chi-kwadrat:\n", chi2_results.head(10))

    # Analiza wpływu tagów na ceny
    tag_price_df = tag_price_analysis(df)
    print("Średnia cena dla poszczególnych tagów:\n", tag_price_df)

    # Wizualizacja danych
    fig1 = px.imshow(tag_matrix, text_auto=True, title="Współwystępowanie tagów (Tabela krzyżowa)")
    fig2 = px.bar(tag_price_df, x='Tag', y='Average Price', title="Średnia cena w zależności od tagu", labels={'Tag': 'Tag', 'Average Price': 'Średnia cena'})
    
    fig1.show()
    fig2.show()

    return {
        "tag_matrix": tag_matrix,
        #"chi2_results": chi2_results,
        "tag_price_analysis": tag_price_df
    }


In [ ]:
# Analiza
analyze_data(df)


In [13]:
fig = px.histogram(df, x='price', nbins=20, title='Rozkład cen gier')
fig.update_layout(xaxis_title='Cena', yaxis_title='Liczba gier')
fig.show()


In [14]:
from collections import Counter

# Liczenie występowania tagów
tag_counts = Counter(tag for tags in df['tags'] for tag in tags)
tag_df = pd.DataFrame(tag_counts.items(), columns=['Tag', 'Count']).sort_values('Count', ascending=False)

# Wizualizacja
fig = px.bar(tag_df.head(20), x='Tag', y='Count', title='Najpopularniejsze tagi')
fig.show()


In [19]:
fig = px.box(df, x='tag_count', y='price', title='Rozkład cen w zależności od liczby tagów', labels={'tag_count': 'Liczba tagów', 'price': 'Cena (zł)'})
fig.show()



In [ ]:
# Tabela krzyżowa
all_tags = list(set(tag for tags in df['tags'] for tag in tags))
cross_tab = pd.DataFrame(0, index=all_tags, columns=all_tags)

for tags in df['tags']:
    for tag1 in tags:
        for tag2 in tags:
            if tag1 != tag2:
                cross_tab.loc[tag1, tag2] += 1

# Wybór najpopularniejszych tagów (np. top 20)
popular_tags = tag_df['Tag'].head(20).tolist()
filtered_cross_tab = cross_tab.loc[popular_tags, popular_tags]

# Wizualizacja
fig = px.imshow(filtered_cross_tab, text_auto=True, title='Współwystępowanie najpopularniejszych tagów', labels={'color': 'Liczba współwystąpień'})
fig.show()


In [ ]:
tag_prices = []

for tag in all_tags:
    subset = df[df['tags'].apply(lambda tags: tag in tags)]
    avg_price = subset['price'].mean()
    tag_prices.append({'Tag': tag, 'Average Price': avg_price})

tag_price_df = pd.DataFrame(tag_prices).sort_values('Average Price', ascending=False)


# Wybór tagów z największą średnią ceną i różnicą cen
top_expensive_tags = tag_price_df.nlargest(10, 'Average Price')
top_cheap_tags = tag_price_df.nsmallest(10, 'Average Price')
filtered_tag_price_df = pd.concat([top_expensive_tags, top_cheap_tags])

# Wizualizacja
fig = px.bar(filtered_tag_price_df, x='Tag', y='Average Price', title='Wpływ wybranych tagów na cenę', labels={'Tag': 'Tag', 'Average Price': 'Średnia cena'})
fig.show()


In [23]:
%pip install sklearn

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Przygotowanie danych
tag_matrix = pd.DataFrame(0, index=df.index, columns=all_tags)
for i, tags in enumerate(df['tags']):
    tag_matrix.loc[i, tags] = 1

scaler = StandardScaler()
X = scaler.fit_transform(tag_matrix)

# Klasteryzacja
kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

# Wizualizacja klastrów (kolor + centroidy)
fig = px.scatter(df, x='price', y='tag_count', color='cluster', title='Klasteryzacja gier (cena vs liczba tagów)', labels={'price': 'Cena (zł)', 'tag_count': 'Liczba tagów'}, hover_data=['title'])
fig.show()


ModuleNotFoundError: No module named 'sklearn'